In [7]:
import pandas as pd
import numpy as np 
from pandas import DataFrame,Series
import os
import re
from datetime import datetime 

1.聚合size()用法

In [ ]:
encoding = 'latin1'

unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
mnames = ['movie_id', 'title', 'genres']

users = pd.read_table(r'c/ch02/movielens/users.dat', sep='::', header=None, names=unames, encoding=encoding,engine = 'python')
ratings = pd.read_table(r'c/ch02/movielens/ratings.dat', sep='::', header=None, names=rnames, encoding=encoding,engine = 'python')
movies = pd.read_table(r'c/ch02/movielens/movies.dat', sep='::', header=None, names=mnames, encoding=encoding,engine = 'python')

筛选电影评论大于250的电影那女不同的评分

In [2]:
data = pd.merge(pd.merge(ratings, users), movies)
mean_ratings = data.pivot_table('rating', index='title',columns='gender', aggfunc='mean')
ratings_by_title = data.groupby('title').size()
active_titles = ratings_by_title.index[ratings_by_title >= 250]
mean_ratings = mean_ratings.ix[active_titles]

找出评分分歧最大的电影

In [4]:
rating_std_by_title=data.groupby('title')['rating'].std()

2.拼接 concat用法 

In [5]:
years = range(1880, 2011)

pieces = []
columns = ['name', 'sex', 'births']

for year in years:
    path = 'c/ch02/names/yob%d.txt' % year
    frame = pd.read_csv(path, names=columns)
    frame['year'] = year
    pieces.append(frame)
# pieces 列表内含有1880-2010年数据 pieces[0]=1880年数据

# Concatenate everything into a single DataFrame
names = pd.concat(pieces, ignore_index=True)
# pd.concat 是按行把多个数组组合到一起，ignore_index=True ,不保留原有的行号

In [6]:
names.groupby(['year', 'sex'])

3.聚合apply  插入一列births的婴儿数相对于总出生的比例 按year和sex分组

In [ ]:
def add_prop(group):
    # 比例除法，需要调整类型
    births = group.births.astype(float) # astype 类型转换 

    group['prop'] = births / births.sum()
    return group
names = names.groupby(['year', 'sex']).apply(add_prop)

In [ ]:
第一种方案
def get_top1000(group):
    return group.sort_values(by='births', ascending=False)[:1000]
grouped = names.groupby(['year', 'sex'])
top1000 = grouped.apply(get_top1000)

In [ ]:
第二种方案
pieces = []
for year, group in names.groupby(['year', 'sex']):
    pieces.append(group.sort_index(by='births', ascending=False)[:1000])
top1000 = pd.concat(pieces, ignore_index=True)

4.生成数组 ()运用不同生成维度不同

In [11]:
arr = np.ones((8, 4))
arr

array([[ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.]])

In [10]:
arr1 = np.ones(4)
arr1

array([ 1.,  1.,  1.,  1.])

In [12]:
arr[1]

array([ 1.,  1.,  1.,  1.])

In [13]:
arr[1,2]

1.0

In [14]:
arr[[1,2]]

array([[ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.]])

In [20]:
x = np.array([[1., 2., 3.], [4., 5., 6.]])
x

array([[ 1.,  2.,  3.],
       [ 4.,  5.,  6.]])

5.花式索引

In [15]:
arr=np.arange(32).reshape((8,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19],
       [20, 21, 22, 23],
       [24, 25, 26, 27],
       [28, 29, 30, 31]])

In [16]:
arr[[1, 5, 7, 2], [0, 3, 1, 2]]#取某行某列

array([ 4, 23, 29, 10])

In [18]:
arr[[1, 5, 7, 2]][:,[0, 3, 1, 2]]

array([[ 4,  7,  5,  6],
       [20, 23, 21, 22],
       [28, 31, 29, 30],
       [ 8, 11,  9, 10]])

In [ ]:
arr[[1, 5, 7, 2]][:, [0, 3, 1, 2]]#取某行且列重排

In [19]:
arr[[1, 5, 7, 2]]

array([[ 4,  5,  6,  7],
       [20, 21, 22, 23],
       [28, 29, 30, 31],
       [ 8,  9, 10, 11]])

6.df.ix 重新索引

In [22]:
frame = DataFrame(np.arange(9).reshape((3, 3)), index=['a', 'c', 'd'],
                  columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [24]:
frame.ix[['a', 'b', 'c', 'd'], ['Texas', 'Utah', 'California']] #ix重新索引

,Texas,Utah,California
a,1.0,NaN,2.0
b,NaN,NaN,NaN
c,4.0,NaN,5.0
d,7.0,NaN,8.0


7.df.apply()

In [25]:
frame = DataFrame(np.random.randn(4, 3), columns=list('bde'),
                  index=['Utah', 'Ohio', 'Texas', 'Oregon'])

In [28]:
frame

,b,d,e
Utah,0.073572,-0.628542,-1.509920
Ohio,-0.464213,0.109784,-0.599720
Texas,1.377319,0.749709,-0.245268
Oregon,0.727776,0.725670,0.354496


In [27]:
f = lambda x: x.max() - x.min()
frame.apply(f) 

b    1.841532
d    1.378251
e    1.864415
dtype: float64

In [30]:
frame.apply(f,axis=1)

Utah      1.583492
Ohio      0.709505
Texas     1.622587
Oregon    0.373280
dtype: float64

In [31]:
def f(x):
    return Series([x.min(), x.max()], index=['min', 'max'])
frame.apply(f)

,b,d,e
min,-0.464213,-0.628542,-1.509920
max,1.377319,0.749709,0.354496


In [32]:
format = lambda x: '%.2f' % x
frame.applymap(format) 

,b,d,e
Utah,0.07,-0.63,-1.51
Ohio,-0.46,0.11,-0.60
Texas,1.38,0.75,-0.25
Oregon,0.73,0.73,0.35


In [33]:
frame['e'].map(format)

Utah      -1.51
Ohio      -0.60
Texas     -0.25
Oregon     0.35
Name: e, dtype: object

In [34]:
data = DataFrame({'Qu1': [1, 3, 4, 3, 4],
                  'Qu2': [2, 3, 1, 2, 3],
                  'Qu3': [1, 5, 2, 4, 4]})
data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


In [35]:
data.apply(pd.value_counts).fillna(0)

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0


In [36]:
data['Qu1'].value_counts()

4    2
3    2
1    1
Name: Qu1, dtype: int64

8.逐块读取文件 对文件逐块迭代

In [4]:
pd.read_csv('c\ch06\ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [2]:
chunker = pd.read_csv('c\ch06\ex6.csv', chunksize=1000)
chunker

In [3]:
tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(),fill_value=0)

In [5]:
tot=tot.sort_values(ascending=False)
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

9.层次化索引merge

In [6]:
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [7]:
pd.merge(left, right, on=['key1'], how='outer')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [8]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


#### 10. 轴向拼接

In [9]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [10]:
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [11]:
s1 = Series([0, 1], index=['a', 'b'])
s3 = Series([5, 6], index=['f', 'g'])
s4 = pd.concat([s1 * 5, s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [12]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [13]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,5


In [14]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


In [16]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [18]:
pd.concat([s1, s1, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,0.0,NaN
b,1.0,1.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


#### 11 df.to_records timestamp

In [3]:
%pwd

'C:\\Users\\ASUS\\Documents\\python-Analysis-master'

In [4]:
data = pd.read_csv('c/ch07/macrodata.csv')
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
data = DataFrame(data.to_records(),
                 columns=pd.Index(['realgdp', 'infl', 'unemp'], name='item'),
                 index=periods.to_timestamp('D', 'end'))
# data.to_records() df转为array 
ldata = data.stack().reset_index().rename(columns={0: 'value'})
wdata = ldata.pivot('date', 'item', 'value')

In [5]:
data.head()

item,realgdp,infl,unemp
date,,,
1959-03-31,2710.349,0.00,5.8
1959-06-30,2778.801,2.34,5.1
1959-09-30,2775.488,2.74,5.3
1959-12-31,2785.204,0.27,5.6
1960-03-31,2847.699,2.31,5.2


In [7]:
ldata.head()

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340


In [11]:
ldata.unstack('item').head()

date  0    1959-03-31 00:00:00
      1    1959-03-31 00:00:00
      2    1959-03-31 00:00:00
      3    1959-06-30 00:00:00
      4    1959-06-30 00:00:00
dtype: object

In [8]:
wdata.head())

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


In [12]:
ldata['value2'] = np.random.randn(len(ldata))
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8 -0.185197 -0.496362 -0.022626
1959-06-30  2.34  2778.801   5.1  1.004538  0.843647  1.115873
1959-09-30  2.74  2775.488   5.3 -0.496557 -1.016607 -0.628388
1959-12-31  0.27  2785.204   5.6 -0.250307  0.972309 -1.545178
1960-03-31  2.31  2847.699   5.2  0.403423  1.128567 -1.777949

In [13]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8 -0.185197 -0.496362 -0.022626
1959-06-30  2.34  2778.801   5.1  1.004538  0.843647  1.115873
1959-09-30  2.74  2775.488   5.3 -0.496557 -1.016607 -0.628388
1959-12-31  0.27  2785.204   5.6 -0.250307  0.972309 -1.545178
1960-03-31  2.31  2847.699   5.2  0.403423  1.128567 -1.777949
1960-06-30  0.14  2834.390   5.2 -0.272993  0.063197  0.663335
1960-09-30  2.70  2839.022   5.6 -2.232055  0.748485  0.207923

#### 12 map 

In [14]:
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami',
                           'corned beef', 'Bacon', 'pastrami', 'honey ham',
                           'nova lox'],
                  'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [15]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [16]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

####  13 pd.cut()

In [2]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, object): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [3]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [4]:
cats.categories

Index(['(18, 25]', '(25, 35]', '(35, 60]', '(60, 100]'], dtype='object')

In [5]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [6]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, object): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [8]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
b=pd.cut(ages, bins, labels=group_names)
b

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [9]:
pd.value_counts(b)

Youth         5
MiddleAged    3
YoungAdult    3
Senior        1
dtype: int64

In [11]:
cats=pd.cut(ages, 4, precision=2)
cats

[(19.96, 30.25], (19.96, 30.25], (19.96, 30.25], (19.96, 30.25], (19.96, 30.25], ..., (30.25, 40.5], (50.75, 61], (40.5, 50.75], (40.5, 50.75], (30.25, 40.5]]
Length: 12
Categories (4, object): [(19.96, 30.25] < (30.25, 40.5] < (40.5, 50.75] < (50.75, 61]]

In [13]:
data = np.random.randn(1000) # 正态分布
cats = pd.qcut(data, 4) # 4等份

In [14]:
pd.value_counts(cats)

(0.684, 3.0217]      250
(-0.0244, 0.684]     250
(-0.684, -0.0244]    250
[-2.956, -0.684]     250
dtype: int64

In [15]:
cats=pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

In [16]:
pd.value_counts(cats)

(-0.0244, 1.333]     400
(-1.278, -0.0244]    400
(1.333, 3.0217]      100
[-2.956, -1.278]     100
dtype: int64

In [17]:
cats

[(-1.278, -0.0244], (-0.0244, 1.333], (-0.0244, 1.333], [-2.956, -1.278], (-1.278, -0.0244], ..., (-1.278, -0.0244], (-1.278, -0.0244], (-0.0244, 1.333], (1.333, 3.0217], (-1.278, -0.0244]]
Length: 1000
Categories (4, object): [[-2.956, -1.278] < (-1.278, -0.0244] < (-0.0244, 1.333] < (1.333, 3.0217]]

#### 14.排列与随机采样

In [3]:
df = DataFrame(np.arange(5 * 4).reshape(5, 4))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [4]:
sampler = np.random.permutation(5)
sampler#df.take(sampler) 不改变源df

array([4, 0, 1, 2, 3])

In [5]:
df.take(sampler)

,0,1,2,3
4,16,17,18,19
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [8]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [6]:
bag = np.array([5, 7, -1, 6, 4])
sampler = np.random.randint(0, len(bag), size=10)# randint(min,max,size)
sampler#df.take(sampler) 改变源df

array([2, 4, 2, 3, 3, 3, 0, 1, 1, 3])

In [7]:
draws = bag.take(sampler)
draws

array([-1,  4, -1,  6,  6,  6,  5,  7,  7,  6])

#### 15哑变量

In [2]:
df = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                'data1': range(6)})
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [3]:
pd.get_dummies(df['key'])

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [4]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [5]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy #join 索引连接 

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


In [6]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('c/ch02/movielens/movies.dat', sep='::', header=None,
                        names=mnames,engine = 'python')
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


** 获取不同的genres**

In [8]:
genres_iter=(set(x.split('|')) for x in movies.genres)
every_genres=set.union(*genres_iter)
every_genres

{'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

** 生成电影类型的哑变量**

In [11]:
dummies = DataFrame(np.zeros((len(movies), len(every_genres))), columns=every_genres)
for i, gen in enumerate(movies.genres):# enumerate 遍历索引和值
    dummies.ix[i, gen.split('|')] = 1
dummies.head()

,Film-Noir,Drama,Horror,Animation,Documentary,Comedy,Musical,Action,Mystery,Western,Fantasy,Romance,Children's,Sci-Fi,Adventure,Crime,Thriller,War
0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.ix[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Film-Noir                                0
Genre_Drama                                    0
Genre_Horror                                   0
Genre_Animation                                1
Genre_Documentary                              0
Genre_Comedy                                   1
Genre_Musical                                  0
Genre_Action                                   0
Genre_Mystery                                  0
Genre_Western                                  0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Children's                               1
Genre_Sci-Fi                                   0
Genre_Adventure                                0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_War           

**pd.get_dummies(pd.qut)**

In [13]:
values = np.random.rand(10)
values

array([ 0.26790042,  0.63738789,  0.12619546,  0.14652243,  0.74422765,
        0.85597383,  0.334792  ,  0.93044579,  0.39647149,  0.2449282 ])

In [14]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
cats=pd.cut(values, bins)
cats

[(0.2, 0.4], (0.6, 0.8], (0, 0.2], (0, 0.2], (0.6, 0.8], (0.8, 1], (0.2, 0.4], (0.8, 1], (0.2, 0.4], (0.2, 0.4]]
Categories (5, object): [(0, 0.2] < (0.2, 0.4] < (0.4, 0.6] < (0.6, 0.8] < (0.8, 1]]

In [15]:
pd.get_dummies(pd.cut(values, bins))

,"(0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1]"
0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0
5,0.0,0.0,0.0,0.0,1.0
6,0.0,1.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,1.0
8,0.0,1.0,0.0,0.0,0.0
9,0.0,1.0,0.0,0.0,0.0


#### 16 字符串正则矢量化

In [3]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = Series(data)

In [4]:
data.isnull()

Dave     False
Rob      False
Steve    False
Wes       True
dtype: bool

In [5]:
data.contains('gmail')

AttributeError: 'Series' object has no attribute 'contains'

In [6]:
data.str.contains('gmail')

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

In [8]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [10]:
a=data.str.findall(pattern, flags=re.IGNORECASE)
a

Dave     [(dave, google, com)]
Rob        [(rob, gmail, com)]
Steve    [(steve, gmail, com)]
Wes                        NaN
dtype: object

In [13]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

D:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: In future versions of pandas, match will change to always return a bool indexer.
  if __name__ == '__main__':


Dave     (dave, google, com)
Rob        (rob, gmail, com)
Steve    (steve, gmail, com)
Wes                      NaN
dtype: object

In [15]:
matches.str.get(1)

Dave     google
Rob       gmail
Steve     gmail
Wes         NaN
dtype: object

In [16]:
matches.str[1]

Dave     google
Rob       gmail
Steve     gmail
Wes         NaN
dtype: object

In [17]:
data.str[:5]

Dave     dave@
Rob      rob@g
Steve    steve
Wes        NaN
dtype: object

In [18]:
matches = data.str.extract(pattern, flags=re.IGNORECASE,expand=True)
matches

,0,1,2
Dave,dave,google,com
Rob,rob,gmail,com
Steve,steve,gmail,com
Wes,NaN,NaN,NaN


In [20]:
import json
db = json.load(open('c/ch07/foods-2011-10-03.json'))
db[0]

{'description': 'Cheese, caraway',
 'group': 'Dairy and Egg Products',
 'id': 1008,
 'manufacturer': '',
 'nutrients': [{'description': 'Protein',
   'group': 'Composition',
   'units': 'g',
   'value': 25.18},
  {'description': 'Total lipid (fat)',
   'group': 'Composition',
   'units': 'g',
   'value': 29.2},
  {'description': 'Carbohydrate, by difference',
   'group': 'Composition',
   'units': 'g',
   'value': 3.06},
  {'description': 'Ash', 'group': 'Other', 'units': 'g', 'value': 3.28},
  {'description': 'Energy',
   'group': 'Energy',
   'units': 'kcal',
   'value': 376.0},
  {'description': 'Water',
   'group': 'Composition',
   'units': 'g',
   'value': 39.28},
  {'description': 'Energy', 'group': 'Energy', 'units': 'kJ', 'value': 1573.0},
  {'description': 'Fiber, total dietary',
   'group': 'Composition',
   'units': 'g',
   'value': 0.0},
  {'description': 'Calcium, Ca',
   'group': 'Elements',
   'units': 'mg',
   'value': 673.0},
  {'description': 'Iron, Fe',
   'group': 

### 17 groupby 迭代

In [2]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,1.629507,-1.035870,a,one
1,-0.427320,-0.102636,a,two
2,-1.654750,0.043124,b,one
3,1.147424,1.220954,b,two
4,1.637620,-0.909654,a,one


In [3]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':       data1     data2 key1 key2
 0  1.629507 -1.035870    a  one
 1 -0.427320 -0.102636    a  two
 4  1.637620 -0.909654    a  one, 'b':       data1     data2 key1 key2
 2 -1.654750  0.043124    b  one
 3  1.147424  1.220954    b  two}

In [4]:
people = DataFrame(np.random.randn(5, 5),
                   columns=['a', 'b', 'c', 'd', 'e'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.ix[2:3, ['b', 'c']] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-0.356751,-0.908496,0.951806,1.269709,-0.061062
Steve,-0.651118,3.423782,0.208789,0.565384,0.552274
Wes,-1.728487,NaN,NaN,-0.801588,1.612161
Jim,0.307705,0.607638,1.435075,-0.739086,0.656521
Travis,1.709205,-0.081654,-0.020616,-0.880428,-1.386347


In [5]:
people.groupby(len).mean()

,a,b,c,d,e
3,-0.592511,-0.150429,1.193441,-0.090322,0.735873
5,-0.651118,3.423782,0.208789,0.565384,0.552274
6,1.709205,-0.081654,-0.020616,-0.880428,-1.386347


In [6]:
grouped = df.groupby('key1')
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.064940,0.933234
b,2.802174,1.177829


In [8]:
grouped['data1'].agg([('fp','mean'),('gd',peak_to_peak)])
#给运用的函数重命名

,fp,gd
key1,,
a,0.946602,2.064940
b,-0.253663,2.802174


In [9]:
grouped['data1'].agg(['mean',peak_to_peak])

,mean,peak_to_peak
key1,,
a,0.946602,2.064940
b,-0.253663,2.802174


In [10]:
fu=['mean','max','min']
grouped.agg(fu)
#不同列运用相同函数聚合

data1                        data2                    
          mean       max      min      mean       max       min
key1                                                           
a     0.946602  1.637620 -0.42732 -0.682720 -0.102636 -1.035870
b    -0.253663  1.147424 -1.65475  0.632039  1.220954  0.043124

In [11]:
grouped.agg({'data1':'mean','data2':'max'})

,data1,data2
key1,,
a,0.946602,-0.102636
b,-0.253663,1.220954


In [12]:
df.groupby(['key1','key2'],as_index=False).mean()

,key1,key2,data1,data2
0,a,one,1.633563,-0.972762
1,a,two,-0.427320,-0.102636
2,b,one,-1.654750,0.043124
3,b,two,1.147424,1.220954


In [16]:
k1_means=df.groupby('key1').mean().add_prefix('mean_')
k1_means

,mean_data1,mean_data2
key1,,
a,0.946602,-0.682720
b,-0.253663,0.632039


In [19]:
pd.merge(df,k1_means,left_on='key1',right_index=True)

,data1,data2,key1,key2,mean_data1,mean_data2
0,1.629507,-1.035870,a,one,0.946602,-0.682720
1,-0.427320,-0.102636,a,two,0.946602,-0.682720
4,1.637620,-0.909654,a,one,0.946602,-0.682720
2,-1.654750,0.043124,b,one,-0.253663,0.632039
3,1.147424,1.220954,b,two,-0.253663,0.632039


In [21]:
df.groupby('key1').transform(np.mean)

,data1,data2
0,0.946602,-0.682720
1,0.946602,-0.682720
2,-0.253663,0.632039
3,-0.253663,0.632039
4,0.946602,-0.682720


In [22]:
def demean(arr):
    return arr-arr.mean()
df.groupby('key1').transform(demean)

,data1,data2
0,0.682904,-0.353150
1,-1.373922,0.580084
2,-1.401087,-0.588915
3,1.401087,0.588915
4,0.691018,-0.226934


### 20 groupby.apply

In [4]:
tips = pd.read_csv('c/ch08/tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


In [25]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [26]:
tips.groupby('smoker').apply(top)
#apply使top函数在df各个片段上调用，结果在由pandas.concat组装

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [29]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [27]:
tips.groupby(['smoker','day']).apply(top,n=1,column='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                  tip_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

In [28]:
tips.groupby('smoker')['tip_pct'].apply(lambda x :x.describe())

smoker       
No      count    151.000000
        mean       0.159328
        std        0.039910
        min        0.056797
        25%        0.136906
        50%        0.155625
        75%        0.185014
        max        0.291990
Yes     count     93.000000
        mean       0.163196
        std        0.085119
        min        0.035638
        25%        0.106771
        50%        0.153846
        75%        0.195059
        max        0.710345
Name: tip_pct, dtype: float64

In [ ]:
移动加权平均数

In [2]:
df = DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                'data': np.random.randn(8),
                'weights': np.random.rand(8)})
df

,category,data,weights
0,a,0.082850,0.257599
1,a,-0.838867,0.877615
2,a,-0.036986,0.489209
3,a,-1.640950,0.867016
4,b,-1.471576,0.101158
5,b,1.333870,0.903104
6,b,-0.536299,0.312575
7,b,-0.714034,0.087474


In [3]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a   -0.865237
b    0.587954
dtype: float64

###  21 qcut 分位数、cut桶分析与groupby 结合

In [31]:
frame = DataFrame({'data1': np.random.randn(1000),
                   'data2': np.random.randn(1000)})
factor = pd.cut(frame.data1, 4)
factor[:10]

0    (-1.847, -0.167]
1     (-0.167, 1.514]
2      (1.514, 3.194]
3    (-1.847, -0.167]
4    (-1.847, -0.167]
5     (-0.167, 1.514]
6    (-1.847, -0.167]
7     (-0.167, 1.514]
8     (-0.167, 1.514]
9     (-0.167, 1.514]
Name: data1, dtype: category
Categories (4, object): [(-3.535, -1.847] < (-1.847, -0.167] < (-0.167, 1.514] < (1.514, 3.194]]

In [33]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(factor)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.535, -1.847]",30.0,2.241329,0.307501,-1.636931
"(-1.847, -0.167]",412.0,2.757605,0.058572,-2.686991
"(-0.167, 1.514]",488.0,3.501560,0.062902,-3.002533
"(1.514, 3.194]",70.0,1.869741,-0.074800,-2.781604


In [35]:
grouping=pd.qcut(frame.data1,10)
grouping[:10]

0     (-0.544, -0.269]
1     (-0.0378, 0.231]
2       (1.282, 3.194]
3     (-0.544, -0.269]
4     (-0.544, -0.269]
5     (-0.0378, 0.231]
6      [-3.528, -1.26]
7    (-0.269, -0.0378]
8       (0.813, 1.282]
9    (-0.269, -0.0378]
Name: data1, dtype: category
Categories (10, object): [[-3.528, -1.26] < (-1.26, -0.831] < (-0.831, -0.544] < (-0.544, -0.269] ... (0.231, 0.486] < (0.486, 0.813] < (0.813, 1.282] < (1.282, 3.194]]

In [36]:
grouping=pd.qcut(frame.data1,10,labels=False)
grouping[:10]
#返回为分位数编号

0    3
1    5
2    9
3    3
4    3
5    5
6    0
7    4
8    8
9    4
Name: data1, dtype: int64

In [37]:
frame.data2.groupby(grouping).apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.241329,0.261960,-2.502741
1,100.0,2.484070,0.062348,-2.447509
2,100.0,1.994190,0.123675,-1.979133
3,100.0,2.757605,-0.021324,-2.686991
4,100.0,2.311554,0.004924,-2.259995
5,100.0,2.612303,0.143283,-1.812438
6,100.0,3.501560,0.009562,-2.192156
7,100.0,3.145955,0.000049,-3.002533
8,100.0,3.161037,0.090678,-2.379853


22 groupby 应用于缺失值填充

In [38]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          1.739186
New York     -0.974461
Vermont            NaN
Florida       2.226948
Oregon        0.663736
Nevada             NaN
California    0.422457
Idaho              NaN
dtype: float64

In [40]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          1.739186
New York     -0.974461
Vermont       0.997224
Florida       2.226948
Oregon        0.663736
Nevada        0.543097
California    0.422457
Idaho         0.543097
dtype: float64

In [41]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])

data.groupby(group_key).apply(fill_func)

Ohio          1.739186
New York     -0.974461
Vermont       0.500000
Florida       2.226948
Oregon        0.663736
Nevada       -1.000000
California    0.422457
Idaho        -1.000000
dtype: float64

### 22 透视表pivot_table()

In [5]:
tips.pivot_table(index=['sex', 'smoker'])

size       tip   tip_pct  total_bill
sex    smoker                                          
Female No      2.592593  2.773519  0.156921   18.105185
       Yes     2.242424  2.931515  0.182150   17.977879
Male   No      2.711340  3.113402  0.160669   19.791237
       Yes     2.500000  3.051167  0.152771   22.284500

In [6]:
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'day'],columns='smoker')

tip_pct                size          
smoker             No       Yes        No       Yes
sex    day                                         
Female Fri   0.165296  0.209129  2.500000  2.000000
       Sat   0.147993  0.163817  2.307692  2.200000
       Sun   0.165710  0.237075  3.071429  2.500000
       Thur  0.155971  0.163073  2.480000  2.428571
Male   Fri   0.138005  0.144730  2.000000  2.125000
       Sat   0.162132  0.139067  2.656250  2.629630
       Sun   0.158291  0.173964  2.883721  2.600000
       Thur  0.165706  0.164417  2.500000  2.300000